<div id="TOP" />
<a href="#END" > END of Notebook (filter) </a>

In [1]:
# -- CONFIG ------------------------------------
LAB_TAG=${PWD##*/} 
LAB_NAME="TFI_Azure_Lab8b. Importing Foreign Resources"
LAB_WEIGHT=82
LAB_MODE="Terraform"
# ----------------------------------------------

# NB=""; cd ~/nb/2021-Terraform/content/tf-labs/XXXX; set +x
DIE() { echo "[$PWD] Error: $*"; banner "ERROR"; exit 1; }
NBTOOL_RC=~/scripts/nbtool.rc; [ ! -f $NBTOOL_RC ] && DIE "No such script as $NBTOOL_RC"
echo "==== Reading nbtool functions from '$NBTOOL_RC' ===="; . $NBTOOL_RC
EXCL_FN_INIT_NOTEBOOK
## --INCLUDE--SECTION--
## --EXCLUDE--SECTION--

==== Reading nbtool functions from '/home/student/scripts/nbtool.rc' ====
TF-1.3.7 -rw-rw-r-- 1 student student 9 Jan  8 19:18 /home/student/.environment
Reading package lists... 0%Reading package lists... 100%Reading package lists... Done
Building dependency tree... 0%Building dependency tree... 0%Building dependency tree... 50%Building dependency tree... 50%Building dependency tree       
Reading state information... 0%Reading state information... 0%Reading state information... Done
sysvbanner is already the newest version (1:1.0-17fakesync1).
The following packages were automatically installed and are no longer required:
  linux-headers-5.4.0-120 linux-headers-5.4.0-120-generic
  linux-image-5.4.0-120-generic linux-modules-5.4.0-120-generic
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.

******** Tue 10 Jan 2023 04:17:05 PM UTC: Notebook initialized on tf TF-1.3.7

NB_DIR=/home/student/src/mjbright.tf-scenarios-private/ServeU

# --INCLUDE--SECTION--

## Background:

In this lab, we will see how we can import AWS resources which were not created by Terraform itself so that Terraform can be subsequently used to manage these resources.

**Note**: Be careful not to manage resource both *outside* and *inside* Terraform

We will look at importing resources using
- the ```terraform import``` command

Tasks.
- As before, create a new directory lab 8b, and create three files, resources.tf, main.tf and vars.tf.

## Tasks

## 1. Importing a resource using ```terraform import```

### First Create a new lab directory and move to that directory


In [2]:
mkdir -p ~/labs/lab8b

cd       ~/labs/lab8b

Use the command ```/usr/local/bin/get_instances.sh -ir``` to determine if there are already running instances.

If there are any you may wish to return to previous lab directories and perform ```terraform destroy``` to delete these resources.

The important thing is to know what instances if any are currently running

You should have no Terraform managed resources in the current workspace, verify this

#### First we will check if there are any instances running

Use the command ```/usr/local/bin/get_instances.sh -ir``` to determine if there are already running instances.

If there are any you may wish to return to previous lab directories and perform ```terraform destroy``` to delete these resources.

The important thing is to know what instances if any are currently running

You should have no Terraform managed resources in the current workspace, verify this

In [3]:
/usr/local/bin/get_instances.sh -ir

Instance data written to /home/student/tmp/get_instances/get_instances.1616672.json

--[count:0]-- All running instances -----------------


: 1

In [4]:
terraform show

#### Launch a new instance using the aws client

Launch a new aws_instance using the aws cli client using the following command:

In [5]:
aws ec2 run-instances --count 1  --tag-specifications \
    'ResourceType=instance,Tags=[{Key=aws-cli,Value=true}]' \
    --image-id ami-0e42deec9aa2c90ce --instance-type t2.micro

{
    "Groups": [],
    "Instances": [
        {
            "AmiLaunchIndex": 0,
            "ImageId": "ami-0e42deec9aa2c90ce",
            "InstanceId": "i-0ba1fb4c98479f149",
            "InstanceType": "t2.micro",
            "LaunchTime": "2023-01-10T16:17:27.000Z",
            "Monitoring": {
                "State": "disabled"
            },
            "Placement": {
                "AvailabilityZone": "us-west-1c",
                "GroupName": "",
                "Tenancy": "default"
            },
            "PrivateDnsName": "ip-172-31-17-147.us-west-1.compute.internal",
            "PrivateIpAddress": "172.31.17.147",
            "ProductCodes": [],
            "PublicDnsName": "",
            "State": {
                "Code": 0,
                "Name": "pending"
            },
            "StateTransitionReason": "",
            "SubnetId": "subnet-01f855549f3efdd85",
            "VpcId": "vpc-0c4ad4047839bc08f",
            "Architecture": "x86_64",
            "BlockD

**Note**: that we added a tag to identify this as a non-Terraform resource

**Note**: You may need to wait a minute for the VM to be visible ( in the running state )

then verify the running instances again using

In [6]:
/usr/local/bin/get_instances.sh

Instance data written to /home/student/tmp/get_instances/get_instances.1617047.json

--[count:2]-- All instances -----------------
{"id":"i-030a9593c957b7aee","image":"ami-0e42deec9aa2c90ce","state":"terminated","Tags":[{"Key":"aws-cli","Value":"true"}],"LaunchTime":"2023-01-10T15:58:08.000Z"}
{"id":"i-0ba1fb4c98479f149","image":"ami-0e42deec9aa2c90ce","state":"pending","Tags":[{"Key":"aws-cli","Value":"true"}],"LaunchTime":"2023-01-10T16:17:27.000Z"}


: 1

In [10]:
/usr/local/bin/get_instances.sh -ir

Instance data written to /home/student/tmp/get_instances/get_instances.1617380.json

--[count:1]-- All running instances -----------------
{"id":"i-0ba1fb4c98479f149","image":"ami-0e42deec9aa2c90ce","state":"running","Tags":[{"Key":"aws-cli","Value":"true"}],"LaunchTime":"2023-01-10T16:17:27.000Z"}


: 1

Again, you should have no Terraform managed resources in the current workspace, verify this


#### Create a resource file for the running instance (not yet managed by Terraform)

We can create a simple terraform resource to be able to import this instance so it can be managed by terraform.

Create a new file import.tf with the **following form** with the **appropriate values** for the instance **you created** earlier:

```hcl
resource "aws_instance" "imported_item" {
    ami = "YOUR_IMAGE_ID"
    instance_type = "YOUR_TYPE"
}
```

In [11]:
#EXCLUDE

cat > import.tf <<EOF
resource "aws_instance" "imported_item" {
    ami           = "ami-0e42deec9aa2c90ce"
    instance_type = "t2.micro"
}
EOF

#. ~/scripts/nbtool.rc

SET_VAR AWS_ID $( /usr/local/bin/get_instances.sh -ir |
                grep -m 1 '"id":' |  sed -e 's/{"id":"//' -e 's/".*//' )
#echo $XX_AWS_ID
#terraform import aws_instance.imported_item $XX_AWS_ID

# TODO: FIX var setting ... do we need XX_VAR_AWS_ID in code cell ??

Instance data written to /home/student/tmp/get_instances/get_instances.1617569.json
Correcting var name 'AWS_ID' to '__AWS_ID'
VAR __AWS_ID="i-0ba1fb4c98479f149"


You can now import your resource - use your instance id from the above output, e.g. *"i-06817c3963d2716c9"* in the command below:

In [12]:
#EXCLUDE

# Let them think for themselves about the terraform init !!

terraform init


Initializing the backend...

Initializing provider plugins...
- Reusing previous version of hashicorp/aws from the dependency lock file
- Using previously-installed hashicorp/aws v4.49.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


In [13]:
terraform import aws_instance.imported_item $__AWS_ID

aws_instance.imported_item: Importing from ID "i-0ba1fb4c98479f149"...
aws_instance.imported_item: Import prepared!
  Prepared aws_instance for import
aws_instance.imported_item: Refreshing state... [id=i-0ba1fb4c98479f149]

Import successful!

The resources that were imported are shown above. These resources are now in
your Terraform state and will henceforth be managed by Terraform.



### Verify that terraform has imported the resource

Now verify that you can see the instance using 
- Looking at the newly created ```terraform.tfstate``` file
- Using the ```terraform show``` and ```terraform state list``` commands

In [14]:
#EXCLUDE

echo terraform show:
terraform show

echo
echo terraform state list:
terraform state list

terraform show:
# aws_instance.imported_item:
resource "aws_instance" "imported_item" {
    ami                                  = "ami-0e42deec9aa2c90ce"
    arn                                  = "arn:aws:ec2:us-west-1:816376574968:instance/i-0ba1fb4c98479f149"
    associate_public_ip_address          = true
    availability_zone                    = "us-west-1c"
    cpu_core_count                       = 1
    cpu_threads_per_core                 = 1
    disable_api_stop                     = false
    disable_api_termination              = false
    ebs_optimized                        = false
    get_password_data                    = false
    hibernation                          = false
    id                                   = "i-0ba1fb4c98479f149"
    instance_initiated_shutdown_behavior = "stop"
    instance_state                       = "running"
    instance_type                        = "t2.micro"
    ipv6_address_count                   = 0
    ipv6_addresses            

### Reflect on this

Think about what we just did, we

- created an *AWS EC2 instance* without using Terraform

- we then created a dummy *import.tf* partially describing the resource, containing just
  - the *ami_id*
  - the *instance_type*
  
- we then ran the ```terraform import``` command referencing the instance *id* of the EC2 instance

Once we performed these steps the resource has now been imported into the terraform state.

We can use this mechanism to start to manage infrastructure resources previously generated outside of *Terraform*.

**NOTE:** Note also that this is not very functional, there are a lot of manual steps to take to import a single resource - but 3rd-party tools exist ...

#### Now we can destroy the instance

You succeeded, you can now cleanup using ```terraform destroy```

Remove, or rename import.tf as import.tf.bak

In [15]:
#EXCLUDE
terraform destroy -auto-approve

aws_instance.imported_item: Refreshing state... [id=i-0ba1fb4c98479f149]

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # aws_instance.imported_item will be destroyed
  - resource "aws_instance" "imported_item" {
      - ami                                  = "ami-0e42deec9aa2c90ce" -> null
      - arn                                  = "arn:aws:ec2:us-west-1:816376574968:instance/i-0ba1fb4c98479f149" -> null
      - associate_public_ip_address          = true -> null
      - availability_zone                    = "us-west-1c" -> null
      - cpu_core_count                       = 1 -> null
      - cpu_threads_per_core                 = 1 -> null
      - disable_api_stop                     = false -> null
      - disable_api_termination              = false -> null
      - ebs_optimized                        = false -> null
      -

### Verify that the terraform state is empty

Check the ```terraform.tfstate``` file and run ```terraform state list``` to verify that we have now destroyed the resource.

### Note that independently of Terraform the VM instance is nevertheless present in AWS but ```Terminating```

Note that the ```/usr/local/bin/get_instances.sh -ir``` command shows ```Running``` instances only ... the previously destroyed instance may still be present but in the ```Terminated``` or other state-*

You can verify none ```Running``` instances using either of the commands
- ```/usr/local/bin/get_instances.sh```
- or ```aws ec2 describe-instances```


In [16]:
/usr/local/bin/get_instances.sh

Instance data written to /home/student/tmp/get_instances/get_instances.1619160.json

--[count:2]-- All instances -----------------
{"id":"i-030a9593c957b7aee","image":"ami-0e42deec9aa2c90ce","state":"terminated","Tags":[{"Key":"aws-cli","Value":"true"}],"LaunchTime":"2023-01-10T15:58:08.000Z"}
{"id":"i-0ba1fb4c98479f149","image":"ami-0e42deec9aa2c90ce","state":"terminated","Tags":[{"Key":"aws-cli","Value":"true"}],"LaunchTime":"2023-01-10T16:17:27.000Z"}


: 1

## 2. Importing resources using ```GoogleCloudPlatform/terraformer```


Terraformer is a tool from Google for importing resources from various providers.

See the list at https://github.com/GoogleCloudPlatform/terraformer#table-of-contents

The tool terraformer-aws has already been installed for you as ``` /usr/local/bin/terraformer-aws```

#### Create again an aws instance using the command

First verify that no instances are running:

In [17]:
/usr/local/bin/get_instances.sh -ir

Instance data written to /home/student/tmp/get_instances/get_instances.1619254.json

--[count:0]-- All running instances -----------------


: 1

In [18]:
aws ec2 run-instances --count 1 --tag-specifications \
    'ResourceType=instance,Tags=[{Key=aws-cli,Value=true}]' \
    --image-id ami-0e42deec9aa2c90ce --instance-type t2.micro

{
    "Groups": [],
    "Instances": [
        {
            "AmiLaunchIndex": 0,
            "ImageId": "ami-0e42deec9aa2c90ce",
            "InstanceId": "i-001f556148d07a147",
            "InstanceType": "t2.micro",
            "LaunchTime": "2023-01-10T16:19:31.000Z",
            "Monitoring": {
                "State": "disabled"
            },
            "Placement": {
                "AvailabilityZone": "us-west-1c",
                "GroupName": "",
                "Tenancy": "default"
            },
            "PrivateDnsName": "ip-172-31-19-124.us-west-1.compute.internal",
            "PrivateIpAddress": "172.31.19.124",
            "ProductCodes": [],
            "PublicDnsName": "",
            "State": {
                "Code": 0,
                "Name": "pending"
            },
            "StateTransitionReason": "",
            "SubnetId": "subnet-01f855549f3efdd85",
            "VpcId": "vpc-0c4ad4047839bc08f",
            "Architecture": "x86_64",
            "BlockD

### Install Terraformer



In [19]:
wget -qO terraformer-all https://github.com/GoogleCloudPlatform/terraformer/releases/download/0.8.22/terraformer-all-linux-amd64

In [20]:
chmod +x terraformer-all
ls -alh terraformer-all

-rwxrwxr-x 1 student student 380M Sep 20 09:30 terraformer-all


In [21]:
./terraformer-all version

Terraformer v0.8.22


**Note:** These update instructions were made following the ```blog post``` at 

https://betterprogramming.pub/terraformer-converting-infrastructure-into-reusable-terraform-code-afe543ad0b15

### Import the new instance using ```terraformer-aws```

The ```terraformer-aws``` usage is documented at https://github.com/GoogleCloudPlatform/terraformer/blob/master/docs/aws.md

We will now use ```terraformer-aws``` to import all ```Running``` instances from the ```us-west-1``` region which we're using.

**Note:** It may also detect the previous ```Terminated``` instance as below, it will import only ```Running``` resources.

In [23]:
./terraformer-all import aws --resources=ec2_instance \
    --connect=true --regions=us-west-1 --profile ""

2023/01/10 16:22:55 aws importing region us-west-1
2023/01/10 16:22:56 aws importing... ec2_instance
2023/01/10 16:22:58 aws done importing ec2_instance
2023/01/10 16:22:58 Number of resources for service ec2_instance: 3
2023/01/10 16:22:58 Refreshing state... aws_instance.tfer--i-030a9593c957b7aee_
2023/01/10 16:22:58 Refreshing state... aws_instance.tfer--i-001f556148d07a147_
2023/01/10 16:22:58 Refreshing state... aws_instance.tfer--i-0ba1fb4c98479f149_
2023/01/10 16:22:59 ERROR: Read resource response is null for resource aws_instance.tfer--i-0ba1fb4c98479f149_
2023/01/10 16:22:59 ERROR: Read resource response is null for resource aws_instance.tfer--i-030a9593c957b7aee_
2023/01/10 16:23:01 ERROR: Unable to refresh resource tfer--i-030a9593c957b7aee_
2023/01/10 16:23:01 ERROR: Unable to refresh resource tfer--i-0ba1fb4c98479f149_
2023/01/10 16:23:01 Filtered number of resources for service ec2_instance: 1
2023/01/10 16:23:01 aws Connecting.... 
2023/01/10 16:23:01 aws save ec2_insta

In [25]:
ls -alhtr

total 380M
-rwxrwxr-x  1 student student 380M Sep 20 09:30 terraformer-all
drwxrwxr-x 13 student student 4.0K Jan 10 15:56 ..
-rw-r--r--  1 student student 1.4K Jan 10 15:59 .terraform.lock.hcl
-rw-rw-r--  1 student student    0 Jan 10 16:01 .LineWrapped.ipynb
-rw-rw-r--  1 student student  119 Jan 10 16:18 import.tf
-rw-rw-r--  1 student student 4.5K Jan 10 16:19 terraform.tfstate.backup
-rw-rw-r--  1 student student  180 Jan 10 16:19 terraform.tfstate
drwxrwxr-x  3 student student 4.0K Jan 10 16:23 generated
drwxrwxr-x  3 student student 4.0K Jan 10 16:23 .


In [26]:
ll -tr generated/aws/ec2_instance

total 28
-rwxrwxr-x 1 student student  124 Jan 10 16:23 provider.tf
-rwxrwxr-x 1 student student  114 Jan 10 16:23 outputs.tf
-rwxrwxr-x 1 student student 1751 Jan 10 16:23 instance.tf
drwxrwxr-x 3 student student 4096 Jan 10 16:23 ..
-rwxrwxr-x 1 student student 5983 Jan 10 16:23 terraform.tfstate
drwxrwxr-x 2 student student 4096 Jan 10 16:23 .


In [27]:
cat generated/aws/ec2_instance/provider.tf

provider "aws" {
  region = "us-west-1"
}

terraform {
	required_providers {
		aws = {
	    version = "~> 4.49.0"
		}
  }
}


In [28]:
cd generated/aws/ec2_instance

In [29]:
terraform init


Initializing the backend...
╷
│ Error: Invalid legacy provider address
│ 
│ This configuration or its associated state refers to the unqualified provider
│ "aws".
│ 
│ You must complete the Terraform 0.13 upgrade process before upgrading to
│ later versions.
╵



: 1

In [33]:
cp -a provider.tf provider.tf.terraformer

cat > provider.tf <<EOF
provider "aws" {
  region = "us-west-1"
}

terraform {
  required_providers {
    aws = {
      source = "hashicorp/aws"
      version = "~> 4.49.0"
    }
  }
}
EOF

In [34]:
terraform init


Initializing the backend...
╷
│ Error: Invalid legacy provider address
│ 
│ This configuration or its associated state refers to the unqualified provider
│ "aws".
│ 
│ You must complete the Terraform 0.13 upgrade process before upgrading to
│ later versions.
╵



: 1

### Verify that the instance has been created

```Terraformer``` has created a subdirectory with the appropriate Terraform config files and a ```terraform.tfstate``` file.

This was a much nicer experience than using ```terraform import``` directly.

In [21]:
cd generated/aws/ec2_instance

terraform state list

aws_instance.tfer--i-094b2ad1b98336c11_


### Browse the documentation

This is quite a powerful tool, facilitating the importing of resources into ```Terraform```

- discovery of resources by type, by region

- importing of resources from many provider types, not just AWS.

Take a look at the documentation at https://github.com/GoogleCloudPlatform/terraformer, 

### Delete the instance has been created

You will need to be in the ```generated/aws/ec2_instance``` directory of course to perform the ```terraform destroy```

In [25]:
#EXCLUDE
ip a
hostname
pwd
#TF_DESTROY

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP group default qlen 1000
    link/ether 8a:72:ed:76:16:d6 brd ff:ff:ff:ff:ff:ff
    inet 192.168.0.10/24 brd 192.168.0.255 scope global dynamic eth0
       valid_lft 23890sec preferred_lft 23890sec
    inet6 2a01:e0a:257:80e0:8872:edff:fe76:16d6/64 scope global dynamic mngtmpaddr noprefixroute 
       valid_lft 86064sec preferred_lft 86064sec
    inet6 fe80::8872:edff:fe76:16d6/64 scope link 
       valid_lft forever preferred_lft forever
3: br-b4ed6895ae45: <NO-CARRIER,BROADCAST,MULTICAST,UP> mtu 1500 qdisc noqueue state DOWN group default 
    link/ether 02:42:d1:f1:2d:81 brd ff:ff:ff:ff:ff:ff
    inet 17


# **Stretch Goal**: Import other resource types


Try now to import other resource types, such as an S3 bucket, using *terraform import*' and/or '*terraforming*' tools

<hr/>
<!-- ![](../../../static/images/LOGO_v2_CROPPED.jpg) --> <img src="../../../static/images/LOGO_v2_CROPPED.jpg" width="200" />

# --EXCLUDE--SECTION--

## **NOTE:** Save notebook first

<div id="END" /> <a href="#TOP" > TOP OF PAGE </a>  <!-- must appear after EXCLUDE pragma !! -->

In [50]:
NB_DIR=~/nb/2021-Terraform/content/tf-labs/8b.ImportingResources/
. ~/scripts/nbtool.rc
pwd
EXCL_FN_FILTER_NOTEBOOK $NB

/home/student/nb/2021-Terraform/content/tf-labs/8b.ImportingResources

==== Filtering current notebook ====
-- . ~/scripts/nbtool.rc

-- /usr/bin/python3 ~/scripts/nbtool.py -f README.ipynb
cells to include[#31]=[[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 20, 21, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]]
cells to include[#31]=[[37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 24, 23, 21, 20, 18, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3]]
-rw-r--r-- 1 student docker 23655 Jan 14 14:25 ~/nb/2021-Terraform/content/tf-labs/8b.ImportingResources//README.ipynb.filtered.ipynb

Good: No VAR defs/usages in i/p file

Good: No VAR defs/usages in o/p file

-- jupyter-nbconvert --to markdown README.ipynb.filtered.ipynb
[NbConvertApp] Converting notebook README.ipynb.filtered.ipynb to markdown
[NbConvertApp] Writing 18292 bytes to README.ipynb.filtered.md

-rw-r--r--  1 student docker    70 Jan 14 14:20 .dir
-rw-r--r--  1 student docker 23655 Jan 14 14:25 .ipynb
-rw-r